Copyright (c) 2023 Matteo Pietro Pillitteri  
<s314404@studenti.polito.it>
<br>
https://github.com/Matteo-Pietro-Pillitteri/Computational-Intelligence

The following code show a comparison between two different kind of hill climbing: random mutation & steepest-steps
<br>
I will publish in future another version of this comparision considering all the single state methods.
<br>
The two methods are compared on the number of evaluations and best-fitness

In [103]:
from itertools import product
from random import random, randint, shuffle, seed, choice
from copy import copy
from functools import reduce
import numpy as np
from scipy import sparse

In [104]:
def make_set_covering_problem(num_points, num_sets, density):
    seed(num_points*2654435761+num_sets+density) #the seed() method is used to initialize the random number generator.
    sets = sparse.lil_array((num_sets, num_points), dtype=bool)
    for s, p in product(range(num_sets), range(num_points)):    
        if random() < density:
            sets[s, p] = True
    for p in range(num_points):
        sets[randint(0, num_sets-1), p] = True
        """ this last for add random noise, for each poin in num_poins we take sets[random s, p] and we assign it True"""
    return sets

Some comments:
<br>
* make set covering problem returns a sparse array where rows are sets and colums are the covered items
* scipy.lil_array((M,N), [dtype]) creates an empy matrix with shape(M, N) and dtype = bool
* product() makes the cartesian product of the arguments and it is equivalent to a nested for-loop

# Halloween Challenge

Find the best solution with the fewest calls to the fitness functions for:
(fewest calls means not the number of updates of the current state, but the number of EVALUATIONS. That is, how many times you need to calculate the "fitness)

* `num_points = [100, 1_000, 5_000]`
* `num_sets = num_points`
* `density = [.3, .7]` 

In [105]:
num_points = 100
num_sets = 100
density = .7
evaluation = 0

In [106]:
x = make_set_covering_problem(num_points, num_sets, density)
print("Element at row=1 and column=2:", x[1, 2])
matrix = x.todense() #the todense method make x dense in order to memorize also False element.
# in a sparse matrix only the elements that are not null are stored in order to save memory

Element at row=1 and column=2: True


Each row is a vector which each element can be covered(true) or not(false)

In [107]:
state = [True, False, False, False, False, False, False, False, False, False]
e = [matrix[row] for row, t in enumerate(state) if t]
print(e)
np.logical_or(np.array([False for _ in range(num_points)]), e)


[array([ True, False,  True,  True,  True,  True,  True,  True,  True,
        True,  True, False, False, False,  True,  True,  True,  True,
        True, False,  True,  True,  True, False,  True,  True,  True,
        True, False,  True,  True, False,  True,  True,  True, False,
       False,  True, False,  True,  True,  True,  True,  True,  True,
       False,  True,  True,  True, False, False, False, False,  True,
        True, False, False,  True,  True, False,  True,  True, False,
       False,  True,  True, False,  True, False,  True, False,  True,
       False,  True,  True,  True,  True,  True, False,  True, False,
        True, False,  True,  True,  True, False,  True, False, False,
        True, False,  True,  True,  True,  True,  True,  True, False,
       False])]


array([[ True, False,  True,  True,  True,  True,  True,  True,  True,
         True,  True, False, False, False,  True,  True,  True,  True,
         True, False,  True,  True,  True, False,  True,  True,  True,
         True, False,  True,  True, False,  True,  True,  True, False,
        False,  True, False,  True,  True,  True,  True,  True,  True,
        False,  True,  True,  True, False, False, False, False,  True,
         True, False, False,  True,  True, False,  True,  True, False,
        False,  True,  True, False,  True, False,  True, False,  True,
        False,  True,  True,  True,  True,  True, False,  True, False,
         True, False,  True,  True,  True, False,  True, False, False,
         True, False,  True,  True,  True,  True,  True,  True, False,
        False]])

# Hill climbing
* `random mutation`

In [108]:


def hill_climbing_evaluate(state):
    global evaluation
    evaluation += 1
    cost=sum(state)
    valid = np.sum(
        reduce(
            np.logical_or,
            [matrix[row] for row, t in enumerate(state) if t],
            np.array([False for _ in range(num_points)])
        )
    )
    
    return valid, -cost

def tweak(state):
    new_state = copy(state)
    index = randint(0, num_points - 1) #random index
    new_state[index] = not new_state[index] #the new state is the same of the current one but with a flipped value (the element in the index position) 
    #Important note: the previous operation means that we move the SETS[I] to taken to not taken!
    return new_state

In [109]:
current_state = [choice([False for _ in range(num_points)]) for _ in range(num_sets)]
#print("current_state: ", current_state)
print(hill_climbing_evaluate(current_state))

global evaluation
evaluation = 0

for step in range(50): 
   
    new_state = tweak(current_state)
    if hill_climbing_evaluate(new_state) >= hill_climbing_evaluate(current_state):
        current_state = new_state
        #print("current_state: ", current_state)
        print(hill_climbing_evaluate(current_state))
        

print("Num of evaluation: ", evaluation)


(0, 0)
(77, -1)
(95, -2)
(100, -3)
Num of evaluation:  103


In [110]:
def tweak_steepest(state, index):
    new_state = copy(state)
    new_state[index] = not new_state[index] 
    return new_state

* `steepest-step`

In [111]:
current_state = [choice([False for _ in range(num_points)]) for _ in range(num_sets)]
local_best =  [choice([False for _ in range(num_points)]) for _ in range(num_sets)]
print("current_state: ", current_state)
print(hill_climbing_evaluate(current_state))

global evaluation
evaluation = 0;
for step in range(50): 
  
    for i in range(num_sets):
        new_local_state = tweak_steepest(current_state, i)
        if hill_climbing_evaluate(new_local_state) >= hill_climbing_evaluate(local_best):
            local_best = new_local_state
            #print("local_best: ", local_best)
            print(hill_climbing_evaluate(local_best))
            
    current_state = local_best
    #print("current_state: ", current_state)
    print(hill_climbing_evaluate(current_state))

print("Num of evaluation: ", evaluation)
    

current_state:  [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]
(0, 0)
(66, -1)
(72, -1)
(72, -1)
(77, -1)
(77, -1)
(79, -1)
(80, -1)
(83, -1)
(83, -1)
(93, -2)
(94, -2)
(95, -2)
(96, -2)
(96, -2)
(99, -2)
(99, -2)
(99, -2)
(99, -2)
(99, -2)
(100, -3)
(100, -3)
(100, -3)
(100, -3)
(100, -3)
(100, -3)
(100, -3)
(100, -3)
(100, -3)
(100, -3)
(100,

<table>
    <tr>
        <th> method </th>
        <th> num points </th>
        <th> num sets   </th>
        <th> evaluations </th>
        <th> iteration (steps) </th>
        <th> best-fitness </th>
        <th> density </th>
    </tr>
    <tr>
        <th> random mutation </th>
        <th> 100 </th>
        <th> 100 </th>
        <th> 110 </th>
        <th> 50 </th>
        <th> (100, -10) </th>
        <th> .3 </th>
    </tr>
     <tr>
        <th> random mutation </th>
        <th> 100 </th>
        <th> 100 </th>
        <th> 103 </th>
        <th> 50 </th>
        <th> (100, -3) </th>
        <th> .7 </th>
    </tr>
      <tr>
        <th> random mutation </th>
        <th> 1000 </th>
        <th> 1000 </th>
        <th> 118 </th>
        <th> 50 </th>
        <th> (1000, -18) </th>
        <th> .3 </th>
    </tr>
     <tr>
        <th> random mutation </th>
        <th> 1000 </th>
        <th> 1000 </th>
        <th> 106 </th>
        <th> 50 </th>
        <th> (1000, -6) </th>
        <th> .7 </th>
    </tr>
    <tr>
        <th> random mutation </th>
        <th> 5000 </th>
        <th> 5000 </th>
        <th>  124</th>
        <th> 50 </th>
        <th> (5000, -24) </th>
        <th> .3 </th>
    </tr>
    <tr>
        <th> random mutation </th>
        <th> 5000 </th>
        <th> 5000 </th>
        <th> 69 </th>
        <th> 30 </th>
        <th> (5000, -9) </th>
        <th> .7 </th>
    </tr>
    <tr>
        <th> steepest steps </th>
        <th> 100 </th>
        <th> 100 </th>
        <th> 10085 </th>
        <th> 50 </th>
        <th> (100, -6) </th>
        <th> .3 </th>
    </tr>
    <tr>
        <th> steepest steps </th>
        <th> 100 </th>
        <th> 100 </th>
        <th> 10141 </th>
        <th> 50 </th>
        <th> (100, -3) </th>
        <th> .7 </th>
    </tr>
     <tr>
        <th> steepest steps </th>
        <th> 1000 </th>
        <th> 1000 </th>
        <th> 100228 </th>
        <th> 50 </th>
        <th> (1000, -10) </th>
        <th> .3 </th>
    </tr>
     <tr>
        <th> steepest steps </th>
        <th> 1000 </th>
        <th> 1000 </th>
        <th> 100158 </th>
        <th> 50 </th>
        <th> (1000, -4) </th>
        <th> .7 </th>
    </tr>
     <tr>
        <th> steepest steps </th>
        <th> 5000 </th>
        <th> 5000 </th>
        <th> 500202 </th>
        <th> 50 </th>
        <th> (5000, -13) </th>
        <th> .3 </th>
    </tr>
    <tr>
        <th> steepest steps </th>
        <th> 5000 </th>
        <th> 5000 </th>
        <th> 300301 </th>
        <th> 30 </th>
        <th> (5000, -5) </th>
        <th> .7 </th>
    </tr>
</table>

